In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'

In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2020-07-24
2020-07-25
2020-07-22
2020-07-23


In [7]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                      2020-07-24            2020-07-25           2020-07-22  \
Product                       HS        MS          HS        MS         HS   
SA                                                                            
Fuels- Bargarh(R)      79148.060  35804.67   75558.150  22824.56   87222.83   
Fuels-Bhawanipatna(R  100737.950  44706.72   94119.660  32859.00   96436.44   
Fuels-JEYPORE(R)      143278.100  66587.43  107657.130  40195.61  134895.26   
Fuels-Jharsuguda(R)   105291.880  34178.19  103398.040  21275.22   97993.80   
Fuels-ROURKELA(R)     280507.700  33828.76  269888.340  33402.66  316304.33   
Fuels-SAMBALPUR (R)   114771.750  25908.89  125161.200  19261.40  120290.02   
Keonjhar I RSA        115044.714  15381.97  125629.518   8563.87   87804.07   
Keonjhar II RSA        13784.780   6084.89   17227.530   4488.35   12213.20   

                                2020-07-23             
Product                     MS          HS         MS  
SA                

In [8]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [9]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-07-24' 'Unnamed: 2' '2020-07-25' 'Unnamed: 4'
 '2020-07-22' 'Unnamed: 6' '2020-07-23' 'Unnamed: 8']


In [10]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-07-25', 'Unnamed: 4', '2020-07-24', 'Unnamed: 2', '2020-07-23', 'Unnamed: 8', '2020-07-22', 'Unnamed: 6']


In [11]:
df = df[cols]
print(df)

             Unnamed: 0 2020-07-25 Unnamed: 4 2020-07-24 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    75558.1    22824.6    79148.1    35804.7   
3  Fuels-Bhawanipatna(R    94119.7      32859     100738    44706.7   
4      Fuels-JEYPORE(R)     107657    40195.6     143278    66587.4   
5   Fuels-Jharsuguda(R)     103398    21275.2     105292    34178.2   
6     Fuels-ROURKELA(R)     269888    33402.7     280508    33828.8   
7   Fuels-SAMBALPUR (R)     125161    19261.4     114772    25908.9   
8        Keonjhar I RSA     125630    8563.87     115045      15382   
9       Keonjhar II RSA    17227.5    4488.35    13784.8    6084.89   

  2020-07-23 Unnamed: 8 2020-07-22 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    95059.7    41983.3    87222.8    43237.2  
3    77081.1    33702.9   

In [12]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [44]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)


2020-07-25

2020-07-24

2020-07-23

2020-07-22

Fuels- Bargarh(R)


In [45]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"

In [46]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [47]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

ioc\00504802
Toptal@22


In [48]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [61]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    This is an auto-generated mail.
    </body>
    </html>
    """
    return html

In [62]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()

75558.15
22824.56
79148.06
35804.67
95059.69
41983.3
87222.83
43237.23
94119.66
100737.95
44706.72
77081.12
33702.86
96436.44
48369.35
107657.13
40195.61
143278.1
66587.43
101034.32
47604.81
134895.26
67461.87
103398.04
21275.22
105291.88
34178.19
89493.18
26049.73
97993.8
30450.23
269888.34
33402.66
280507.7
33828.76
322542.88
35294.5
316304.33
36306.3
125161.2
19261.4
114771.75
25908.89
121215.37
22271.72
120290.02
23419.36
125629.52
8563.87
115044.71
15381.97
113298.82
14346.96
87804.07
15939.81
17227.53
4488.35
13784.78
6084.89
13601.47
4985.27
12213.2
6336.09
